## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [15]:
import json
with open("jawiki-country.json", 'r') as wiki:
    for article in wiki.readlines():
        data = json.loads(article)
        if data["title"] == "イギリス":
            break
    
gb_text = data["text"]
with open("gb_text", 'w') as out:
    out.write(gb_text)

#print(gb_text)

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

### memo
categoryは一番下の方に書いてある

In [22]:
import re

with open("gb_text", 'r') as f:
    for line in f.readlines():
        match = re.search("Category", line)
        if match:
            print(line)

[[Category:イギリス|*]]

[[Category:英連邦王国|*]]

[[Category:G8加盟国]]

[[Category:欧州連合加盟国]]

[[Category:海洋国家]]

[[Category:君主国]]

[[Category:島国|くれいとふりてん]]

[[Category:1801年に設立された州・地域]]


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

### memo
- 角カッコはバックスラッシュが必要
- `(?P<name> 正規表現)`で名前からアクセス可能

In [26]:
import re
with open("gb_text", 'r') as f:
    for line in f.readlines():
        match = re.search("\[\[Category:(?P<name>.*)\]\]", line)
        if match:
            print(match.group('name'))

イギリス|*
英連邦王国|*
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国|くれいとふりてん
1801年に設立された州・地域


## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [55]:
import re
with open("gb_text", 'r') as f:
    for line in f.readlines():
        match = re.search("(?P<section>\=+)(?P<name>.*)(?P=section)$", line)
        if match:
            print("Level:%d title:%s" % (len(match.group('section'))-1, match.group('name')))

Level:1 title:国名
Level:1 title:歴史
Level:1 title:地理
Level:2 title:気候
Level:1 title:政治
Level:1 title:外交と軍事
Level:1 title:地方行政区分
Level:2 title:主要都市
Level:1 title:科学技術
Level:1 title:経済
Level:2 title:鉱業
Level:2 title:農業
Level:2 title:貿易
Level:2 title:通貨
Level:2 title:企業
Level:1 title:交通
Level:2 title:道路
Level:2 title:鉄道
Level:2 title:海運
Level:2 title:航空
Level:1 title:通信
Level:1 title:国民
Level:2 title:言語
Level:2 title:宗教
Level:2 title: 婚姻 
Level:2 title:教育
Level:1 title:文化
Level:2 title:食文化
Level:2 title:文学
Level:2 title: 哲学 
Level:2 title:音楽
Level:3 title:イギリスのポピュラー音楽
Level:2 title:映画
Level:2 title:コメディ
Level:2 title:国花
Level:2 title:世界遺産
Level:2 title:祝祭日
Level:1 title:スポーツ
Level:2 title:サッカー
Level:2 title:競馬
Level:2 title:モータースポーツ
Level:1 title:脚注
Level:1 title:関連項目
Level:1 title:外部リンク


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

### memo
(File or ファイル): (ファイル名)| (情報) |... 

In [43]:
import re
with open("gb_text", 'r') as f:
    for line in f.readlines():
        match = re.search("[File | ファイル]:(?P<name>.*\.[a-zA-Z]+)\|", line)
        if match:
            print(match.group('name'))

Royal Coat of Arms of the United Kingdom.svg
Battle of Waterloo 1815.PNG
The British Empire.png
Uk topo en.jpg
BenNevis2005.jpg
Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg
Palace of Westminster, London - Feb 2007.jpg
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
Scotland Parliament Holyrood.jpg
London.bankofengland.arp.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow T5.jpg
Anglospeak.svg
CHANDOS3.jpg
The Fabs.JPG
PalaceOfWestminsterAtNight.jpg
Westminster Abbey - West Door.jpg
Edinburgh Cockburn St dsc06789.jpg
Canterbury Cathedral - Portal Nave Cross-spire.jpeg
Kew Gardens Palm House, London - July 2009.jpg
2005-06-27 - United Kingdom - England - London - Greenwich.jpg
Stonehenge2007 07 30.jpg
Yard2.jpg
Durham Kathedrale Nahaufnahme.jpg
Roman Baths in Bath Spa, England - July 2006.jpg
Fountains Abbey vie

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

### MEMO
|から始まって = で繋がれてるのを探す  
wikitableを弾くために｜と=の間（フィールド名）に |があるものは除外

In [77]:
import re

basic_info = {}
with open("gb_text", 'r') as f:
    for line in f.readlines():
        match = re.match("\|(?P<field>[^|]+)=(?P<value>.+)$", line)
        if match:
            basic_info[match.group('field')] = match.group('value')

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ．

### memo
2,3,5つのシングルクォーテーション囲いを消す

In [80]:
import re

basic_info = {}
with open("gb_text", 'r') as f:
    for line in f.readlines():
        match = re.match("\|(?P<field>[^|]+)=(?P<value>.+)$", line)
        if match:
            no_emph = re.sub("\'", "", match.group('value'))
            basic_info[match.group('field')] = no_emph
            
basic_info

{'略名 ': ' イギリス',
 '日本語国名 ': ' グレートブリテン及び北アイルランド連合王国',
 '公式国名 ': ' {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>',
 '国旗画像 ': ' Flag of the United Kingdom.svg',
 '国章画像 ': ' [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク ': ' （[[イギリスの国章|国章]]）',
 '標語 ': ' {{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）',
 '国歌 ': ' [[女王陛下万歳|神よ女王陛下を守り給え]]',
 '位置画像 ': ' Location_UK_EU_Europe_001.svg',
 '公用語 ': ' [[英語]]（事実上）',
 '首都 ': ' [[ロンドン]]',
 '最大都市 ': ' ロンドン',
 '元首等肩書 ': ' [[イギリスの君主|女王]]',
 '元首等氏名 ': ' [[エリザベス2世]]',
 '首相等肩書 ': ' [[イギリスの首相|首相]]',
 '首相等氏名 ': ' [[デーヴィッド・キャメロン]]',
 '面積順位 ': ' 76',
 '面積大きさ ': ' 1 E11',
 '面積値 ': ' 244,820',
 '水面積率 ': ' 1.3%',
 '人口統計年 ': ' 2011',
 '人口順位 ': ' 22',
 '人口大きさ ': ' 1 E7',
 '人口値 ': ' 63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口密度値 ': ' 246',
 'GDP統計

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ．

### memo
[[記事名]]	 or [[記事名|表示文字]] or [[記事名#節名|表示文字]]

表示と異なる可能性がある|の前半を最小マッチングで検出して弾く

In [97]:
import re

basic_info = {}
with open("gb_text", 'r') as f:
    for line in f.readlines():
        match = re.match("\|(?P<field>[^|]+)=(?P<value>.+)$", line)
        if match:
            no_emph = re.sub("\'", "", match.group('value'))
            no_link = re.sub("\[\[(?P<link>[^|]*?)\|?(?P<display>[^|]+?)\]\]", "\g<display>", no_emph)
            basic_info[match.group('field')] = no_link
            
basic_info

{'略名 ': ' イギリス',
 '日本語国名 ': ' グレートブリテン及び北アイルランド連合王国',
 '公式国名 ': ' {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>',
 '国旗画像 ': ' Flag of the United Kingdom.svg',
 '国章画像 ': ' [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク ': ' （国章）',
 '標語 ': ' {{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）',
 '国歌 ': ' 神よ女王陛下を守り給え',
 '位置画像 ': ' Location_UK_EU_Europe_001.svg',
 '公用語 ': ' 英語（事実上）',
 '首都 ': ' ロンドン',
 '最大都市 ': ' ロンドン',
 '元首等肩書 ': ' 女王',
 '元首等氏名 ': ' エリザベス2世',
 '首相等肩書 ': ' 首相',
 '首相等氏名 ': ' デーヴィッド・キャメロン',
 '面積順位 ': ' 76',
 '面積大きさ ': ' 1 E11',
 '面積値 ': ' 244,820',
 '水面積率 ': ' 1.3%',
 '人口統計年 ': ' 2011',
 '人口順位 ': ' 22',
 '人口大きさ ': ' 1 E7',
 '人口値 ': ' 63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口密度値 ': ' 246',
 'GDP統計年元 ': ' 2012',
 'GDP値元 = 1兆5478億<ref name="imf-statistics-gdp">[htt

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

### memo
ref, br, 波括弧を消す

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

### memo
ヒントが謎